In [97]:
from subprocess import Popen, PIPE, STDOUT, check_output
from lib.InputStreamChunker import InputStreamChunker
from time import sleep
import pandas as pd
import numpy as np
from collections import OrderedDict

In [223]:
def loc_parse(loc_str):
    lines = loc_str.split("\n")
    title = lines[2].strip()
    desc = lines[3]
    other_desc = "\n".join([line.strip() for line in lines[4:]])
    return title, desc, other_desc


def get_data():
    l_data = []
    ch.data_available.wait(0.5)
    sleep(0.1)
    ch.data_unoccupied.clear()
    while ch.data:
        l_data.append(ch.data.pop(0)[:-1])
    ch.data_available.clear()
    ch.data_unoccupied.set()
    return b"\n".join(l_data).decode(encoding="utf-8")


def send_data(data):
    _ = p.stdin.write(data.encode() + b"\n")
    _ = p.stdin.flush()
    return get_data()


def start():
    ch = InputStreamChunker([b"\n"])  # ('\x04')
    ch.daemon = True
    ch.start()
    p = Popen([r'cmd /c dfrotz\dfrotz.exe -m games\Advent.z5'], stdout=ch.input,
              stdin=PIPE, stderr=STDOUT, shell=True)
    return p, ch


def restart():
    global p,ch
    p.terminate()
    del ch
    p,ch = start()
    return p, ch


def look(loc_list, move, move_title, move_desc):
    title, desc, other = loc_parse(send_data("look"))
    loc_it = OrderedDict({
        "move": move,
        "title": title,
        "desc": desc,
        "move_title":move_title,
        "move_desc":move_desc
    })
    loc_list.append(loc_it)


def parse_move_response(s):
    lines = s.split("\n")
    move_title = lines[2]
    move_desc = "\n".join(lines[3:])
    return move_title, move_desc

In [77]:
def find_same_loc_id(dfl, loc_id):
    prev_loc_id = loc_id.shift()
    prev_loc_id[0] = 0
    prev_loc_id = prev_loc_id.astype(int)
    temp_id = dfl["move"] + dfl["title"] + prev_loc_id.astype(str)
    s_loc_id_min = loc_id.groupby(temp_id).min()
    loc_id_new = temp_id.replace(s_loc_id_min)
    return loc_id_new

def find_same_loc_id_back(dfl, loc_id):
    prev_loc_id = loc_id.shift(-1)
    prev_loc_id.iloc[-1] = -1
    prev_loc_id = prev_loc_id.astype(int)
    next_move = dfl["move"].shift(-1)
    temp_id = next_move + dfl["title"] + prev_loc_id.astype(str)
    s_loc_id_min = loc_id.groupby(temp_id).min()
    loc_id_new = temp_id.replace(s_loc_id_min)
    loc_id_new.iloc[-1] = loc_id.iloc[-1]
    loc_id_new[next_move=="start"] = loc_id[next_move=="start"]
    return loc_id_new.astype(int)

In [158]:
def calculate_dfl(loc_list):
    dfl = pd.DataFrame(loc_list)
    loc_id=dfl.index.to_series()
    start_title = loc_list[0]["title"]
    start_desc = loc_list[0]["desc"]
    loc_id[(dfl.title == start_title)&(dfl.desc == start_desc)] = 0
    #1
    loc_id_new = loc_id
    it=0
    while(True):
        loc_id_old = loc_id_new
        loc_id_new = find_same_loc_id(dfl, loc_id_new)
        it += 1
        if (loc_id_old==loc_id_new).all():
            break
    #print(f"Found same locations in {it} iterations")
    loc_id_new_prev = loc_id_new

    #2
    it=0
    while(True):
        loc_id_old = loc_id_new
        loc_id_new = find_same_loc_id_back(dfl, loc_id_new)
        it += 1
        if (loc_id_old==loc_id_new).all():
            break
    #print(f"Found same locations backwards in {it} iterations")
    loc_id_new_back = loc_id_new

    #3
    it=0
    while(True):
        loc_id_old = loc_id_new
        loc_id_new = find_same_loc_id(dfl, loc_id_new)
        it += 1
        if (loc_id_old==loc_id_new).all():
            break
    #print(f"Found same locations forwards2 in {it} iterations")
    loc_id_new_prev2 = loc_id_new
    
    #4
    it=0
    while(True):
        loc_id_old = loc_id_new
        loc_id_new = find_same_loc_id_back(dfl, loc_id_new)
        it += 1
        if (loc_id_old==loc_id_new).all():
            break
    #print(f"Found same locations backwards in {it} iterations")
    loc_id_new_back2 = loc_id_new

    dfl2=dfl
    dfl2["loc_id1"] = loc_id_new_prev
    dfl2["loc_id2"] = loc_id_new_back
    dfl2["loc_id3"] = loc_id_new_prev2
    dfl2["loc_id4"] = loc_id_new_back2
    dfl2["loc_id"] = dfl2.loc_id4
    dfl2["from_start"] = pd.Series(np.ones((len(dfl2,)),dtype=int)).groupby((dfl2.loc_id==0).cumsum()).cumsum()-1
    dfl2["to_start"] = pd.Series(np.ones((len(dfl2,)),dtype=int)).groupby((dfl2.loc_id==0).cumsum()).apply(lambda x: x.iloc[::-1].cumsum().iloc[::-1])
    dfl2["to_reset"] = pd.Series(np.ones((len(dfl2,)),dtype=int)).groupby((dfl2.move=="start").cumsum()).apply(lambda x: x.iloc[::-1].cumsum().iloc[::-1])
    return dfl2    

In [180]:
def moves_multi_titles(dfl):
    titles = dfl.groupby("title")["loc_id"].agg(lambda x: len(set(x)) == 1)
    multi_titles = list(titles.index[~titles])
    multi_title = multi_titles[0]
    multi_title_ids = dfl.loc_id[dfl.title==multi_title].drop_duplicates()
    df1_f=dfl
    df1_f = df1_f[df1_f.loc_id == multi_title_ids.iloc[0]]
    df1_f = df1_f[df1_f.to_start != df1_f.to_reset]
    row = df1_f.iloc[0]
    end_moves = list(dfl.move.loc[row.name+1:row.name+row.to_start])
    df1_f=dfl
    df1_f = df1_f[df1_f.loc_id == multi_title_ids.iloc[1]]
    row = df1_f.iloc[0]
    start_moves = list(dfl.move.loc[row.name-row.from_start+1:row.name])
    moves = start_moves + end_moves
    return moves, multi_title_ids

In [154]:
def explore_moves(moves):
    restart()
    get_data()
    look(loc_list, "start","","")
    for move in moves:
        move_title, move_desc = parse_move_response(send_data(move))
        look(loc_list, move, move_title, move_desc)

In [5]:
p, ch = start()

In [56]:
loc_list = []
l_moves=[
    "nswenswe",
    "snwesnwe",
    "wenswens",
    "ewnsewns",
    "nen",
]
for moves in l_moves:
    restart()
    get_data()
    look(loc_list, "start","","")
    for move in moves:
        move_title, move_desc = parse_move_response(send_data(move))
        look(loc_list, move, move_title, move_desc)
dfl = pd.DataFrame(loc_list)
dfl

,move,title,desc,move_title,move_desc
0,start,At End Of Road,You are standing at the end of a road before a...,,
1,n,In Forest,"You are in open forest, with a deep valley to ...",In Forest,"You are in open forest, with a deep valley t..."
2,s,In Forest,"You are in open forest, with a deep valley to ...",In Forest,
3,w,In Forest,"You are in open forest, with a deep valley to ...",In Forest,
4,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,You are in a valley in the forest beside a s...
5,n,At End Of Road,You are standing at the end of a road before a...,At End Of Road,
6,s,In A Valley,You are in a valley in the forest beside a str...,In A Valley,
7,w,In Forest,You are in open forest near both a valley and ...,In Forest,
8,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,
9,start,At End Of Road,You are standing at the end of a road before a...,,


In [79]:
loc_id=dfl.index.to_series()
start_title = loc_list[0]["title"]
start_desc = loc_list[0]["desc"]
loc_id[(dfl.title == start_title)&(dfl.desc == start_desc)] = 0
#1
loc_id_new = loc_id
it=0
while(True):
    loc_id_old = loc_id_new
    loc_id_new = find_same_loc_id(dfl, loc_id_new)
    it += 1
    if (loc_id_old==loc_id_new).all():
        break
print(f"Found same locations in {it} iterations")
loc_id_new_prev = loc_id_new

#2
it=0
while(True):
    loc_id_old = loc_id_new
    loc_id_new = find_same_loc_id_back(dfl, loc_id_new)
    it += 1
    if (loc_id_old==loc_id_new).all():
        break
print(f"Found same locations backwards in {it} iterations")
loc_id_new_back = loc_id_new

#3
it=0
while(True):
    loc_id_old = loc_id_new
    loc_id_new = find_same_loc_id(dfl, loc_id_new)
    it += 1
    if (loc_id_old==loc_id_new).all():
        break
print(f"Found same locations forwards2 in {it} iterations")
loc_id_new_prev2 = loc_id_new

dfl2=dfl.copy()
dfl2["loc_id1"] = loc_id_new_prev
dfl2["loc_id2"] = loc_id_new_back
dfl2["loc_id3"] = loc_id_new_prev2
dfl2

Found same locations in 3 iterations
Found same locations backwards in 3 iterations
Found same locations forwards2 in 2 iterations


,move,title,desc,move_title,move_desc,loc_id1,loc_id2,loc_id3
0,start,At End Of Road,You are standing at the end of a road before a...,,,0,0,0
1,n,In Forest,"You are in open forest, with a deep valley to ...",In Forest,"You are in open forest, with a deep valley t...",1,1,1
2,s,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,2,2,2
3,w,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,3,1,1
4,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,You are in a valley in the forest beside a s...,4,4,4
5,n,At End Of Road,You are standing at the end of a road before a...,At End Of Road,,0,0,0
6,s,In A Valley,You are in a valley in the forest beside a str...,In A Valley,,6,6,4
7,w,In Forest,You are in open forest near both a valley and ...,In Forest,,7,7,7
8,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,,8,8,8
9,start,At End Of Road,You are standing at the end of a road before a...,,,0,0,0


In [85]:
titles = dfl2.groupby("title")["loc_id3"].agg(lambda x: len(set(x)) == 1)
multi_titles = list(titles.index[~titles])
multi_titles

['In A Valley', 'In Forest']

In [91]:
multi_title = multi_titles[0]
multi_title_ids = dfl2.loc_id3[dfl2.title==multi_title].drop_duplicates()
multi_title_ids

4      4
8      8
23    23
32    32
Name: loc_id3, dtype: int64

In [141]:
dfl2["from_start"] = pd.Series(np.ones((len(dfl2,)),dtype=int)).groupby((dfl2.loc_id3==0).cumsum()).cumsum()-1
dfl2["to_start"] = pd.Series(np.ones((len(dfl2,)),dtype=int)).groupby((dfl2.loc_id3==0).cumsum()).apply(lambda x: x.iloc[::-1].cumsum().iloc[::-1])
dfl2["to_reset"] = pd.Series(np.ones((len(dfl2,)),dtype=int)).groupby((dfl2.move=="start").cumsum()).apply(lambda x: x.iloc[::-1].cumsum().iloc[::-1])
df1_f=dfl2
df1_f = df1_f[df1_f.loc_id3 == multi_title_ids.iloc[0]]
df1_f = df1_f[df1_f.to_start != df1_f.to_reset]
row = df1_f.iloc[0]
end_moves = list(dfl2.move.loc[row.name+1:row.name+row.to_start])
df1_f=dfl2
df1_f = df1_f[df1_f.loc_id3 == multi_title_ids.iloc[1]]
row = df1_f.iloc[0]
start_moves = list(dfl2.move.loc[row.name-row.from_start+1:row.name])
moves = start_moves + end_moves
moves

['s', 'w', 'e', 'n']

In [142]:
restart()
get_data()
look(loc_list, "start","","")
for move in moves:
    move_title, move_desc = parse_move_response(send_data(move))
    look(loc_list, move, move_title, move_desc)

In [155]:
dfl = calculate_dfl(loc_list)
moves = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'n']

In [160]:
explore_moves(moves)

In [166]:
dfl = calculate_dfl(loc_list)
moves = moves_multi_titles(dfl)
moves

['n', 's', 'e', 'n']

In [167]:
explore_moves(moves)

In [176]:
dfl = calculate_dfl(loc_list)
moves = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'e', 'n']

In [177]:
explore_moves(moves)

In [181]:
dfl = calculate_dfl(loc_list)
moves, multi_title_ids = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'e', 'n']

In [184]:
explore_moves(moves)

In [185]:
dfl = calculate_dfl(loc_list)
moves, multi_title_ids = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'n']

In [187]:
explore_moves(moves)

In [188]:
dfl = calculate_dfl(loc_list)
moves, multi_title_ids = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'n']

In [191]:
explore_moves(moves)

In [192]:
dfl = calculate_dfl(loc_list)
moves, multi_title_ids = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'n']

In [195]:
explore_moves(moves)

In [196]:
dfl = calculate_dfl(loc_list)
moves, multi_title_ids = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'n']

In [198]:
explore_moves(moves)

In [199]:
dfl = calculate_dfl(loc_list)
moves, multi_title_ids = moves_multi_titles(dfl)
moves

['n', 's', 'w', 'e', 's', 'w', 'n']

In [200]:
multi_title_ids

4      4
70    70
Name: loc_id, dtype: int32

In [201]:
dfl.loc[:,list(dfl.columns[:5])+["loc_id","to_start","from_start"]]

,move,title,desc,move_title,move_desc,loc_id,to_start,from_start
0,start,At End Of Road,You are standing at the end of a road before a...,,,0,5,0
1,n,In Forest,"You are in open forest, with a deep valley to ...",In Forest,"You are in open forest, with a deep valley t...",1,4,1
2,s,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,1,3,2
3,w,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,1,2,3
4,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,You are in a valley in the forest beside a s...,4,1,4
5,n,At End Of Road,You are standing at the end of a road before a...,At End Of Road,,0,4,0
6,s,In A Valley,You are in a valley in the forest beside a str...,In A Valley,,4,3,1
7,w,In Forest,You are in open forest near both a valley and ...,In Forest,,1,2,2
8,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,,4,1,3
9,start,At End Of Road,You are standing at the end of a road before a...,,,0,2,0


In [203]:
dfl.desc[dfl.title=="In Forest"].unique()

array(['You are in open forest, with a deep valley to one side.\n',
       'You are in open forest near both a valley and a road.\n'], dtype=object)

In [213]:
dfl["next_move"]=dfl.move.shift(-1)
dfl.loc[dfl.index[dfl.desc=='You are in open forest, with a deep valley to one side.\n']+1]

,move,title,desc,move_title,move_desc,loc_id1,loc_id2,loc_id3,loc_id4,loc_id,from_start,to_start,to_reset,next_move
2,s,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,2,2,1,1,1,2,3,7,w
3,w,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,3,1,1,1,1,3,2,6,e
4,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,You are in a valley in the forest beside a s...,4,4,4,4,4,4,1,5,n
34,n,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,34,34,34,34,34,5,2,2,s
35,s,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,35,35,35,35,35,6,1,1,start
36,start,At End Of Road,You are standing at the end of a road before a...,,,0,0,0,0,0,0,3,4,n
53,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,You are in a valley in the forest beside a s...,32,4,4,4,4,3,1,2,n
58,w,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,3,1,1,1,1,3,5,6,e
59,e,In A Valley,You are in a valley in the forest beside a str...,In A Valley,You are in a valley in the forest beside a s...,4,4,4,4,4,4,4,5,s
72,n,In Forest,"You are in open forest, with a deep valley to ...",In Forest,,72,72,72,72,72,8,1,1,start


In [214]:
dfl.title.unique()

array(['At End Of Road', 'In Forest', 'In A Valley', 'At Hill In Road',
       'Inside Building', 'At Slit In Streambed'], dtype=object)

In [218]:
row = dfl[dfl.title=="Inside Building"].iloc[0]

In [221]:
row.desc

'You are inside a building, a well house for a large spring.\n\nThere are some keys on the ground here.\n\nThere is tasty food here.\n\nThere is a shiny brass lamp nearby.\n\nThere is an empty bottle here.\n'

In [220]:
print(row.desc)

You are inside a building, a well house for a large spring.

There are some keys on the ground here.

There is tasty food here.

There is a shiny brass lamp nearby.

There is an empty bottle here.



In [222]:
dfl.desc.loc[88]

'You are in open forest, with a deep valley to one side.\n'